In [1]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing
import wav_to_spec
import os

In [2]:
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES = 200
SOUNDS =np.array(['yes','no','right','five','nine'])
path_out = '/home/josearangos/Documentos/Projects/Voice_Recognition/data/spectrograms'

In [3]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    
    
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    return matrixMajor

In [4]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [5]:
path_word_speakerId.shape

(1000, 3)

In [6]:
path_word_speakerId

array([['data/yes/004ae714_nohash_0.wav', 'yes', '004ae714'],
       ['data/yes/004ae714_nohash_1.wav', 'yes', '004ae714'],
       ['data/yes/00970ce1_nohash_0.wav', 'yes', '00970ce1'],
       ...,
       ['data/nine/106a6183_nohash_4.wav', 'nine', '106a6183'],
       ['data/nine/10ace7eb_nohash_0.wav', 'nine', '10ace7eb'],
       ['data/nine/10ace7eb_nohash_1.wav', 'nine', '10ace7eb']],
      dtype='<U32')

In [7]:
testP = path_word_speakerId[1,0]
testP

'data/yes/004ae714_nohash_1.wav'

In [17]:
speakers =[]
words = []
spectrograms = []

 #Encode etiqueta del path audio,speaker y la palabra
leSpeaker = preprocessing.LabelEncoder()
leWord =  preprocessing.LabelEncoder()

#Encode
leSpeaker.fit(path_word_speakerId[:,2])
leWord.fit(path_word_speakerId[:,1])


for p in path_word_speakerId:
    name = p[0].split('/')[-1].split('.')[0]+'.jpg'
    word = p[1]
    speaker_id = p[-1]
    
 #Transformación del encoder
    labelSpeaker = leSpeaker.transform([speaker_id])
    labelWord= leWord.transform([word])     
    
    spect = wav_to_spec.wav2spect('../'+p[0]) 
    spectrograms.append(spect)
    speakers.append(labelSpeaker[0])
    words.append(labelWord[0])

In [18]:
df = pd.DataFrame([spectrograms,speakers,words],index=['Spectrogram','speaker','word']).T

In [19]:
df

,Spectrogram,speaker,word
0,"[[-5.899411, -6.593098, -21.35734, -21.187449,...",0,4
1,"[[-23.02585, -23.02585, -23.02585, -23.02585, ...",0,4
2,"[[-5.7146406, -6.407505, -20.297499, -20.98756...",1,4
3,"[[-5.8207245, -6.529051, -13.875972, -12.77553...",3,4
4,"[[-5.9325137, -6.628802, -16.829935, -17.02652...",3,4
...,...,...,...
995,"[[-5.8164544, -6.5176325, -15.9848795, -14.700...",114,1
996,"[[-5.7210984, -6.4039636, -15.700572, -17.4116...",114,1
997,"[[-5.596871, -6.3141885, -13.834732, -14.83248...",114,1
998,"[[-6.3125834, -7.004977, -21.878431, -22.00323...",116,1


In [20]:
df.to_hdf('/home/josearangos/Documentos/Projects/Voice_Recognition/data/spectrograms/spectrograms_1000.hd5', key = 'description')

/home/josearangos/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['Spectrogram', 'speaker', 'word']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


Rellenar de cero 

https://www.kaggle.com/alphasis/light-weight-cnn-lb-0-74